In [1]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

plt.style.use("ggplot")

# ---- CONFIG ----
API_KEY = os.getenv("SPRINGER_API_KEY")
QUERY = "cancer"   # <--- change this to any search term

if not API_KEY:
    raise ValueError("❌ SPRINGER_API_KEY is missing. Set it in your environment.")


display(Markdown(f"## 🔍 Searching Springer Open Access for: **{QUERY}**"))


# ---- API CALL ----
url = "https://api.springernature.com/openaccess/json"
params = {"q": QUERY, "api_key": API_KEY, "p": 50}

response = requests.get(url, params=params)
data = response.json()

# Fallback if premium content error
if "error_description" in data:
    display(Markdown("⚠️ Premium content detected — retrying..."))
    response = requests.get(url, params=params)
    data = response.json()

records = data.get("records", [])


# ---- HANDLE ZERO RESULTS ----
if not records:
    display(Markdown(f"### ❌ No open-access results found for **{QUERY}**"))
else:
    display(Markdown(f"### ✅ Found **{len(records)}** open-access results"))


# ---- CONVERT TO DATAFRAME ----
articles = []
for r in records:
    articles.append({
        "Title": r.get("title", ""),
        "Publication": r.get("journalTitle", ""),
        "Year": r.get("onlinePublicationDate", "")[:4] if r.get("onlinePublicationDate") else "",
        "Abstract": r.get("abstract", ""),
        "URL": r.get("url", "")
    })

df = pd.DataFrame(articles)
display(df.head())


# ---- SUMMARY ----
if len(df) > 0:
    most_common_year = df["Year"].value_counts().idxmax()
    top_journal = df["Publication"].value_counts().idxmax()

    display(Markdown(f"""
## 📊 Summary
- **Most common publication year:** {most_common_year}
- **Most common journal:** {top_journal}
- **Total records:** {len(df)}
    """))


# ---- YEAR DISTRIBUTION PLOT ----
if len(df) > 0:
    year_counts = df["Year"].value_counts().sort_index()

    plt.figure(figsize=(10, 4))
    year_counts.plot(kind="bar")
    plt.title(f"Publication Year Distribution for '{QUERY}'")
    plt.xlabel("Year")
    plt.ylabel("Count")
    plt.show()


## 🔍 Searching Springer Open Access for: **cancer**

### ❌ No open-access results found for **cancer**

""
